# 0. Setup

In [ ]:
from google.colab import drive

drive.mount("./drive")

## Language Files

In [ ]:
!mkdir .tgz
!mkdir data

!cp -R ./drive/MyDrive/projects/language-translation/data/* ./.tgz

In [ ]:
import os, tarfile, random
files = os.listdir(".tgz")

for fname in files:
  FOLDER_NAME = fname.split(".")[0]
  FILE_PATH = os.path.join(".tgz", fname)
  tar_file = tarfile.open(FILE_PATH, "r")
  for item in tar_file:
    tar_file.extract(item, os.path.join("data", FOLDER_NAME))
  os.remove(FILE_PATH)

## JSON Files

In [ ]:
!cp ./drive/MyDrive/projects/language-translation/data/*.json .

# Language Detection

In [ ]:
import pandas as pd
import numpy as np

#10000 examples per class
df = pd.read_json("language-detection-10000.json", orient="records", lines=True)
df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

X = df.text.values
y = df.label.values

count_vec = CountVectorizer()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=42)

In [ ]:
X_train = count_vec.fit_transform(X_train)
X_test = count_vec.transform(X_test)

model = MultinomialNB()
model.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train), model.score(X_test, y_test)

In [ ]:
#50000 examples per class
df = pd.read_json("language-detection-50000.json", orient="records", lines=True)

data = (count_vec.transform(df.text.values), df.label.values)
model.score(data[0], data[1])